In [1]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input, Activation
from keras.layers.noise import AlphaDropout
from keras.initializers import lecun_normal
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
X_train = np.load('../data/X_train.npy', mmap_mode = 'r')
y_train = np.load('../data/y_train_k.npy', mmap_mode = 'r')
X_val = np.load('../data/X_val.npy', mmap_mode = 'r')
y_val = np.load('../data/y_val_k.npy', mmap_mode = 'r')
mean = np.mean(X_train, axis = 0)
std = np.std(X_train, axis = 0)
std[std == 0] = 1
X_train = X_train - mean / std
X_val = X_val - mean / std

In [3]:
ins = Input(shape = (X_train.shape[1],))
x = Dense(2048, kernel_initializer=lecun_normal(seed=42))(ins)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(2048, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(1024, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(1024, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(512, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(512, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(256, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(256, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(128, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
x = Dense(128, kernel_initializer=lecun_normal(seed=42))(x)
x = Activation('selu')(x)
#x = AlphaDropout(.05)(x)
outs = Dense(y_train.shape[1], activation = 'softmax')(x)

model = Model(ins, outs)

In [4]:
opt = Adam(lr = .00002)
model.compile(opt, 'categorical_crossentropy', metrics = ['acc'])
stopper = EarlyStopping(monitor='val_acc', patience = 3, mode='auto')
checker = ModelCheckpoint('../src/ensemble/seluModels/selu_test_best_0', monitor='val_acc', mode='auto')

In [5]:
model.fit(X_train, y_train,
          epochs=100, batch_size=32,
          validation_data=(X_val, y_val),
          callbacks = [stopper, checker])

Train on 3428511 samples, validate on 736105 samples
Epoch 1/100
3428511/3428511 [==============================] - 857s - loss: 0.0202 - acc: 0.9972 - val_loss: 0.0137 - val_acc: 0.9980
Epoch 2/100
3428511/3428511 [==============================] - 844s - loss: 0.0139 - acc: 0.9982 - val_loss: 0.0124 - val_acc: 0.9984
Epoch 3/100
3428511/3428511 [==============================] - 845s - loss: 0.0154 - acc: 0.9983 - val_loss: 0.0150 - val_acc: 0.9985
Epoch 4/100
3428511/3428511 [==============================] - 845s - loss: 0.0159 - acc: 0.9984 - val_loss: 0.0153 - val_acc: 0.9984
Epoch 5/100
3428511/3428511 [==============================] - 843s - loss: 0.0152 - acc: 0.9985 - val_loss: 0.0182 - val_acc: 0.9982
Epoch 6/100
3428511/3428511 [==============================] - 845s - loss: 0.0154 - acc: 0.9986 - val_loss: 0.0144 - val_acc: 0.9987
Epoch 7/100
3428511/3428511 [==============================] - 845s - loss: 0.0152 - acc: 0.9986 - val_loss: 0.0195 - val_acc: 0.9983
Epoch 8/1

In [4]:
model = load_model('../src/ensemble/selu_best.h5')

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 122)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              251904    
_________________________________________________________________
activation_1 (Activation)    (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
activation_2 (Activation)    (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
activation_3 (Activation)    (None, 1024)              0         
__________

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
val_preds = model.predict(X_val)
int_preds_val = np.array([np.argmax(r) for r in val_preds])
int_y_val = np.array([np.argmax(r) for r in y_val])
val_acc = accuracy_score(int_y_val, int_preds_val)
print(val_acc)

0.998894179499


In [8]:
train_preds = model.predict(X_train)
int_preds_train = np.array([np.argmax(r) for r in train_preds])
int_y_train = np.array([np.argmax(r) for r in y_train])
train_acc = accuracy_score(int_y_train, int_preds_train)
print(train_acc)

0.998908855769


In [9]:
X_test = np.load('../data/X_test.npy', mmap_mode='r')
y_test = np.load('../data/y_test_k.npy', mmap_mode='r')

In [10]:
test_preds = model.predict(X_test)
int_preds = np.array([np.argmax(r) for r in test_preds])
int_y_test = np.array([np.argmax(r) for r in y_test])
test_acc = accuracy_score(int_y_test, int_preds)
print(test_acc)

0.997804623781


In [11]:
np.save('../data/preds/selu_val_preds', val_preds)
np.save('../data/preds/selu_train_preds', train_preds)
np.save('../data/preds/selu_test_preds', test_preds)